# Algoritmo Genético - Problema da mochila

O problema da mochila consiste em escolher os itens com maior valor (benefício) que podem ser inseridos em uma mochila sem extrapolar uma condição $ C $ (peso, espaço, etc).

Esse é um problema de otimização em que o objetivo é maximizar o benefício dentro da condição $ C $.

Esse tipo de problema pode ser resolvido por meio de um algoritmo genético.

Os algoritmos genéticos mapeiam o problema para um vetor de bits (há outras formas) em que o bit 1 representa que o item foi selecionado e o bit 0 o item não foi selecionado.

O algoritmo gera uma população inicial aleatória que será mapeada para um vetor de bits. Em seguida é aplicada uma função fitness (heurística) sobre a população, os melhores dessa população serão escolhidos para reprodução (*crossover*), então ocorre  uma mutação (altera o valor de nenhum, um ou mais bits) com base em um fator de probabilidade, então a função fitness é aplicada novamente e os mais aptos são selecionados para a nova geração. O processo se repete até atingir uma condição de parada.

Pode ocorrer de um ou mais indivíduos não pertencerem ao conjunto de soluções do problema, então é aplicada um punição ou correção sobre os indivíduos antes de selecionar a nova geração.

In [102]:
import random

# peso máximo da mochila
limite_peso = 120 #C
tamanho_populacao = 10 #Np
probabilidade_mutacao = 0.1 # Pm
probabilidade_crossover = 0.9 #Pc
maximo_geracoes = 500 

# população inicial do problema
itens_disponiveis = [ 
    { 'peso': 3, 'valor': 1},
    { 'peso': 8, 'valor': 3},
    { 'peso': 12, 'valor': 1},
    { 'peso': 2, 'valor': 8},
    { 'peso': 8, 'valor': 9},
    { 'peso': 4, 'valor': 3},
    { 'peso': 4, 'valor': 2},
    { 'peso': 5, 'valor': 8},
    { 'peso': 1, 'valor': 5},
    { 'peso': 1, 'valor': 1},
    { 'peso': 8, 'valor': 1},
    { 'peso': 6, 'valor': 6},
    { 'peso': 4, 'valor': 3},
    { 'peso': 3, 'valor': 2},
    { 'peso': 3, 'valor': 5},
    { 'peso': 5, 'valor': 2},
    { 'peso': 7, 'valor': 3},
    { 'peso': 3, 'valor': 8},
    { 'peso': 5, 'valor': 9},
    { 'peso': 7, 'valor': 3},
    { 'peso': 4, 'valor': 2},
    { 'peso': 3, 'valor': 4},
    { 'peso': 7, 'valor': 5},
    { 'peso': 2, 'valor': 4},
    { 'peso': 3, 'valor': 3},
    { 'peso': 5, 'valor': 1},
    { 'peso': 4, 'valor': 3},
    { 'peso': 3, 'valor': 2},
    { 'peso': 7, 'valor': 14},
    { 'peso': 19, 'valor': 32},
    { 'peso': 20, 'valor': 20},
    { 'peso': 21, 'valor': 19},
    { 'peso': 11, 'valor': 15},
    { 'peso': 24, 'valor': 37},
    { 'peso': 13, 'valor': 18},
    { 'peso': 17, 'valor': 13},
    { 'peso': 18, 'valor': 19},
    { 'peso': 6, 'valor': 10},
    { 'peso': 15, 'valor': 15},
    { 'peso': 25, 'valor': 40},
    { 'peso': 12, 'valor': 17},
    { 'peso': 19, 'valor': 39},
]


In [21]:
def gerar_populacao_inicial():
    num_max_itens = len(itens_disponiveis)
    
    return [ [random.choice([0,1]) for i in range(num_max_itens)] for j in range(tamanho_populacao)] 


def print_geracao(geracao):
    for g in geracao:
        print('\n')
        for gene in g:
            print('%s ' %(gene), end='', flush=True)
        

populacao_inicial = gerar_populacao_inicial()

print('População inicial')
print_geracao(populacao_inicial)

População inicial


0 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 1 1 1 

1 1 1 1 0 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 

1 1 1 1 1 0 1 1 0 0 1 1 0 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 

1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0 1 0 1 0 0 0 1 

0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 

0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 1 

0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1 0 0 

0 1 1 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 0 0 0 1 1 1 

0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 

0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 0 

## Função fitness

É avaliada a heurística da população gerada para futuramente escolher os melhores para reprodução.

In [3]:
# recupera os indices dos valores selecionados para serem inseridos na mochila
def get_index_populacao(populacao):
    return [[index for (index, item) in enumerate(itens_disponiveis) if p[index]] for p in populacao]

# recupera os itens selecionados da população
def get_itens_populacao(populacao):
    return [[item for (index, item) in enumerate(itens_disponiveis) if p[index]] for p in populacao]

# recupera os itens de um unico individuo da população
def get_itens_individuo(individuo):
    return [item for (index, item) in enumerate(itens_disponiveis) if individuo[index]]

# Função fitness para uma população
# Retorna a soma dos pesos e valores de cada item da mochila
def fitness(populacao):
    itens_populacao = get_itens_populacao(populacao)
    
    fitness_values = []
    for itens_individuo in itens_populacao:
        fitness_values.append( sum_itens(itens_individuo) )
        
    return fitness_values

# Função fitness para um unico indivíduo da população
def fitness_individuo(individuo):
    itens_individuo = get_itens_individuo(individuo)
    
    return sum_itens(itens_individuo)

def fitness_normalizada(populacao):
    fitness_values = fitness(populacao)
    fitness_total = sum_fitness(fitness_values)
    normalizado = []
    
    for individuo in populacao:
        f = fitness_individuo(individuo)
        f_normalizado = {
            'peso': f['peso'] / fitness_total['peso'], 
            'valor': f['valor'] / fitness_total['valor']
        }
        
        normalizado.append(f_normalizado)
        
    return normalizado
    
# Soma todos os valores da função fitness apresentados
def sum_fitness(fitness_values):
    return sum_itens(fitness_values)
    

# soma os pesos e valores dos itens apresentados
def sum_itens(itens):
    soma_peso = 0
    soma_valor = 0
    for item in itens:
        soma_peso += item['peso']
        soma_valor += item['valor']
    
    return {'peso': soma_peso, 'valor': soma_valor}
   

fitness_values = fitness(populacao_inicial)
print(fitness_values)

print(fitness_individuo(populacao_inicial[5]))

print(sum_fitness(fitness_values))

print(fitness_normalizada(populacao_inicial))

[{'peso': 168, 'valor': 199}, {'peso': 188, 'valor': 201}, {'peso': 234, 'valor': 279}, {'peso': 193, 'valor': 210}, {'peso': 214, 'valor': 267}, {'peso': 158, 'valor': 161}, {'peso': 176, 'valor': 225}, {'peso': 216, 'valor': 213}, {'peso': 190, 'valor': 204}, {'peso': 215, 'valor': 223}]
{'peso': 158, 'valor': 161}
{'peso': 1952, 'valor': 2182}
[{'peso': 0.0860655737704918, 'valor': 0.09120073327222732}, {'peso': 0.09631147540983606, 'valor': 0.0921173235563703}, {'peso': 0.11987704918032786, 'valor': 0.12786434463794683}, {'peso': 0.09887295081967214, 'valor': 0.09624197983501374}, {'peso': 0.1096311475409836, 'valor': 0.1223648029330889}, {'peso': 0.08094262295081968, 'valor': 0.07378551787351054}, {'peso': 0.09016393442622951, 'valor': 0.10311640696608616}, {'peso': 0.11065573770491803, 'valor': 0.09761686526122823}, {'peso': 0.09733606557377049, 'valor': 0.09349220898258478}, {'peso': 0.11014344262295082, 'valor': 0.10219981668194317}]


## Seleção

Seleção dos indivíduos aptos a se reproduzirem e passar 

In [5]:
def selecao_roleta(populacao):
    p = fitness_normalizada(populacao)
    
    selecionados = []
    for individuo in populacao:
        p_selecao = random.uniform(0, 1)
        
        index_individuo_selecionado = 0
        soma = p[index_individuo_selecionado]['valor']
        while soma < p_selecao:
            index_individuo_selecionado += 1
            soma += p[index_individuo_selecionado]['valor']
        
        selecionados.append(populacao[index_individuo_selecionado])
    
    return selecionados


print_geracao(selecao_roleta(populacao_inicial))    



1 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 1 

1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 0 0 1 0 

1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 

1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 0 0 1 0 

0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 

1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 

1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 0 0 

0 0 1 0 1 1 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 

1 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 1 

1 0 0 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 

# Crossover


In [100]:
def crossover(populacao, probabilidade = probabilidade_crossover):
    
    populacao_crossover = list(populacao) #copia a população inicial para não deletar
    tamanho_populacao = len(populacao_crossover)
    
    num_individuos_crossover = 0 #numero de individuos que passaram por crossover
    crossovered = [] #população após crossover
    
    while num_individuos_crossover < tamanho_populacao:
        num_individuos_crossover += 2
        
        individuo_1 = select_and_remove(populacao_crossover)
        individuo_2 = select_and_remove(populacao_crossover)
        
        # probabilidade de crossover menor que o limiar
        p_not_crossover = random.uniform(0, 1)
        if probabilidade < p_not_crossover:
            # caso não tenha ocorrido o crossover
            crossovered.append(individuo_1)
            crossovered.append(individuo_2)
            continue
        
        ponto_corte  = random.randrange(len(individuo_1))
        
        filho_1 = list(individuo_1[: ponto_corte ]) + list(individuo_2[ponto_corte : ])
        filho_2 = list(individuo_2[: ponto_corte ]) + list(individuo_1[ponto_corte : ])
        
        crossovered.append(filho_1)
        crossovered.append(filho_2)
    
    return crossovered
        
def select_and_remove(populacao):
    index_individuo = random.randrange(len(populacao))
    individuo = populacao[index_individuo]
    
    del populacao[index_individuo]
    
    return individuo

populacao = crossover(populacao_inicial)
print_geracao(populacao)




1 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 

0 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 

0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1 1 1 

0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 1 1 0 1 0 0 

0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 1 

0 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 0 0 0 1 1 1 

0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0 1 0 1 0 0 0 1 

1 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 0 

0 1 0 1 0 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 

1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 1 1 1 

# Mutação

In [103]:
def mutar(populacao, probabilidade=probabilidade_mutacao):
    
    for (index, individuo) in enumerate(populacao):
        populacao[index] = mutar_individuo(individuo, probabilidade)
    
    return populacao


def mutar_individuo(individuo, probabilidade):
    for (index, gene) in enumerate(individuo):
        p_not_mutar = random.uniform(0, 1)
        #não muta
        if(probabilidade < p_not_mutar):
            continue
        
        if individuo[index] == 1:
            individuo[index] = 0
        else:
            individuo[index] = 1
    
    return individuo

print_geracao(mutar(populacao))



1 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 1 1 0 

0 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 

0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 1 1 1 

0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 

1 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 

0 0 0 1 1 1 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 

0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 

0 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1 0 0 1 

0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1 1 1 0 0 1 1 0 1 1 0 1 1 

1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 1 1 1 